**CAPSTONE PROJECT - THE BATTLE OF THE NEIGHBORHOODS**

***INTRODUCTION***

Climbing is a rising activity, gaining popularity as the sport becomes more and more mainstream. Following its introduction in the next olympics in Tokyo, the demand for new climbing gyms is now higher than ever.
This project aims at finding out the best borough in London to open a new climbing gym and could be of interest for anyone looking for a new business opportunity in a young and thriving market.

The Foursquare location data of the existing clymbing gyms in London will be used to figure out the best borough to open a new gym, based on the proximity of the existing gyms and the size of potential customer base.

***DATA***

As already mentioned, this project wil use the Foursquare location data of the existing clymbing gyms in London to figure out the best borough to open a new gym, based on the proximity of the existing gyms and the size of potential customer base. This will be assesed from the following Wikipedia table, which lists all  Greater London's boroughs, their geographical coordinates and population: https://en.wikipedia.org/wiki/List_of_London_boroughs

The best borough will be the one where no gym yet exists and whose five closest boroughs also lack a climbing gym. In case there will be more than one borough fitting this description, the most populous will be chosen, as it will in principle provide the largest customer base. This choice penalises boroughs at the outskirt of Greater London, as they tend to border with fewer boroughs than if they were in a more central position. We would anyway rather open our gym in a more central borough so the criterion chosen shoud still hold a valuable result.

***CODE***

*SCRAPE THE TABLE AND CREATE THE DATAFRAME*

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib.request
import re

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_London_boroughs"
page = urllib.request.urlopen(url)                              #open the url and put the HTML into the page variable
soup = BeautifulSoup(page, "lxml")                              #parse the HTMLinto the BeautifulSoup parse tree format
all_tables=soup.find_all("table")                               #bring back all instances of the 'table' tag in the HTML
right_table=soup.find('table', class_='wikitable sortable')     #retain only the table of interest
right_table

<table class="wikitable sortable" style="font-size:100%" width="100%">
<tbody><tr>
<th>Borough
</th>
<th>Inner
</th>
<th>Status
</th>
<th>Local authority
</th>
<th>Political control
</th>
<th>Headquarters
</th>
<th>Area (sq mi)
</th>
<th>Population (2013 est)<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup>
</th>
<th>Co-ordinates
</th>
<th><span style="background:#67BCD3"> Nr. in map </span>
</th></tr>
<tr>
<td><a href="/wiki/London_Borough_of_Barking_and_Dagenham" title="London Borough of Barking and Dagenham">Barking and Dagenham</a> <sup class="reference" id="cite_ref-2"><a href="#cite_note-2">[note 1]</a></sup>
</td>
<td>
</td>
<td>
</td>
<td><a href="/wiki/Barking_and_Dagenham_London_Borough_Council" title="Barking and Dagenham London Borough Council">Barking and Dagenham London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a href="/wiki/Barking_Town_Hall" title="Barking Town Hall">Town Hal

In [3]:
#loop through the elements (td) of each row (tr) and assign them to the respective column
bor=[]
area=[]
pop=[]
coord=[]

for row in right_table.findAll('tr'):       
    cells=row.findAll('td')
    if len(cells)==10:
        bor.append(cells[0].find(text=True))
        area.append(float(re.sub(r'[^\w\s.]','',cells[6].find(text=True))))     #conversion to float
        pop.append(int(re.sub(r'[^\w\s.]','',cells[7].find(text=True))))        #conversion to integer
        coord.append(cells[8].findAll(text=True))
coord[0]

['51°33′39″N',
 ' ',
 '0°09′21″E',
 '\ufeff / \ufeff',
 '51.5607°N 0.1557°E',
 '\ufeff / ',
 '51.5607; 0.1557',
 '\ufeff (',
 'Barking and Dagenham',
 ')',
 '\n']

In [4]:
#as the coord list is still messy, we clean it up and create a lat and a long list
lat=[]
long=[]
for c in coord:
    geo=[s for s in c if s.startswith('5')][2]                    #exploit the fact that all the latitudes start with '5'
    lat.append(float(re.sub(r'[^\w\s.]','',geo.split(';')[0])))   #conversion to float
    long.append(float(re.sub(r'[^\w\s.]','',geo.split(';')[1])))  #conversion to float

In [5]:
#manual insertion of the data for City of London, which wasn't part of the table
bor.insert(6, 'City of London')
area.insert(6, 1.12)
pop.insert(6, 7000)
lat.insert(6, 51.5155)
long.insert(6, -0.0922)

In [6]:
#convert the column lists into a pandas dataframe
df=pd.DataFrame(list(zip(bor, area, pop, lat, long)), columns =['Borough', 'Area (sqmi)', 'Population', 'Latitude', 'Longitude'])
df

,Borough,Area (sqmi),Population,Latitude,Longitude
0,Barking and Dagenham,13.93,194352,51.5607,0.1557
1,Barnet,33.49,369088,51.6252,0.1517
2,Bexley,23.38,236687,51.4549,0.1505
3,Brent,16.70,317264,51.5588,0.2817
4,Bromley,57.97,317899,51.4039,0.0198
5,Camden,8.40,229719,51.5290,0.1255
6,City of London,1.12,7000,51.5155,-0.0922
7,Croydon,33.41,372752,51.3714,0.0977
8,Ealing,21.44,342494,51.5130,0.3089
9,Enfield,31.74,320524,51.6538,0.0799


VISUALISE THE DIFFERENT BOROUGHS ON THE MAP

In [7]:
import folium
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: ...working... done

# All requested packages already installed.



In [8]:
#create an empty map of London
address = 'London, UK'
geolocator = Nominatim(user_agent="london_explorer")
location = geolocator.geocode(address)
lat_lon = location.latitude
long_lon = location.longitude
london1 = folium.Map(location=[lat_lon, long_lon], zoom_start=10)
london1

In [9]:
#add borough markers to the map
for lat, lng, borough in zip(df['Latitude'], df['Longitude'], df['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(london1)  
    
london1

We can see that all the boroughs, with the exception of City of London, are placed outside London. A quick check on google confirm that the coordinates displayed on Wikipedia are actually wrong. A new dataset with the correct coordinates was then found (https://github.com/naomiggg/the-best-borough-in-london/blob/master/boroughs.json) and loaded as an excel file, and the analysis was resumed.

In [10]:
new=pd.read_excel('C:/Users/marcori94/Documents/Coursera/Python/Capstone project/Coursera/london.xlsx')
new.head()

,{,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,{,NaN,NaN,NaN,NaN
1,"""area""","""Barking and Dagenham"",",NaN,NaN,NaN
2,"""latitude""","51.54527673,",NaN,NaN,NaN
3,"""longitude""","0.13352768,",NaN,NaN,NaN
4,"""population""","209000,",NaN,NaN,NaN


In [11]:
#remove unwanted characters in the '{' column to allow searching for keywords
clean=[]
for c in new['{']:
    b=c.replace('\"','')
    b=b.replace(' ','')
    clean.append(b)
new['clean']=clean
new.head()

,{,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,clean
0,{,NaN,NaN,NaN,NaN,{
1,"""area""","""Barking and Dagenham"",",NaN,NaN,NaN,area
2,"""latitude""","51.54527673,",NaN,NaN,NaN,latitude
3,"""longitude""","0.13352768,",NaN,NaN,NaN,longitude
4,"""population""","209000,",NaN,NaN,NaN,population


In [12]:
#create a new dataframe with the new data
table=pd.DataFrame(df['Borough'])                                             #add the borough's column
key=['population','latitude','longitude']

for h in key:                                                                 #select rows containing pop, lat and long data
    new2=new.loc[new['clean']==h]                                             #and add them to the dataframe
    new2.reset_index(inplace=True)
    table[h]=new2['Unnamed: 1']
    
table['Area (km2)']=df['Area (sqmi)']*2.59                                    #add the area column, convert to square kilometers
table.columns=['Borough', 'Population','Latitude','Longitude','Area (km2)']   #rename the columns
table.head()

,Borough,Population,Latitude,Longitude,Area (km2)
0,Barking and Dagenham,"209000,","51.54527673,","0.13352768,",36.0787
1,Barnet,"389600,","51.5607,","-0.210017107,",86.7391
2,Bexley,"244300,","51.457389,",0.138861,60.5542
3,Brent,"332100,","51.4549,","-0.267820662,",43.2530
4,Bromley,"327900,","51.3719994,","0.051524176,",150.1423


In [13]:
table= table.replace(',','', regex=True)       #remove useless characters
table.dtypes                                  #check the variable type in each column

Borough        object
Population     object
Latitude       object
Longitude      object
Area (km2)    float64
dtype: object

In [14]:
#perform the necessary type conversions
table['Population'] = table['Population'].astype(int)
table['Latitude'] = table['Latitude'].astype(float)
table['Longitude'] = table['Longitude'].astype(float)
table.dtypes  

Borough        object
Population      int32
Latitude      float64
Longitude     float64
Area (km2)    float64
dtype: object

In [15]:
#calculate the population density in each borough and create the final dataframe
table['Population density (people/km2)']=table['Population']/table['Area (km2)']
data=table[['Borough', 'Population density (people/km2)','Latitude','Longitude']]

The final dataset is therefore:

In [16]:
data

,Borough,Population density (people/km2),Latitude,Longitude
0,Barking and Dagenham,5792.891651,51.545277,0.133528
1,Barnet,4491.630649,51.560700,-0.210017
2,Bexley,4034.402238,51.457389,0.138861
3,Brent,7678.080133,51.454900,-0.267821
4,Bromley,2183.928180,51.371999,0.051524
5,Camden,11146.350432,51.546394,-0.157424
6,City of London,3033.645891,51.529000,-0.092171
7,Croydon,4466.560887,51.371400,-0.087157
8,Ealing,6331.758197,51.513000,-0.331026
9,Enfield,4050.769646,51.653800,-0.087272


We can try again to plot the data, and see that the boroughs are now in the correct positions.

In [130]:
london2 = folium.Map(location=[lat_lon, long_lon], zoom_start=10)

for lat, lng, borough in zip(data['Latitude'], data['Longitude'], data['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(london2)  
    
london2

GET THE COORDINATES OF THE CLIMBING GYMS IN EACH BOROUGH FROM FOURSQUARE

In [18]:
import numpy as np
import math

In [19]:
CLIENT_ID = 'NGZZIWUHGMNPKD5TITSCTWNBOZD4IMAPOA3B2VYEJOG11XC1'
CLIENT_SECRET = 'Y0Y3IDAOQR3UWGM5SXAN4WLHZIFMM5UBOLHZCMSSX31WCZX1' 
VERSION = '20180605'

In [20]:
#function to return the climbing gyms in each borough within a radius calculated from the area of the borough
def getNearbyGyms(names, latitudes, longitudes, r):
    
    query='Climbing'
    venues_list=[]
    for name, lat, lng, rad in zip(names, latitudes, longitudes, r):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            lat,
            lng,
            VERSION,
            query,
            rad)

            
        # make the GET request
        results = requests.get(url).json()["response"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'], 
            v['categories'][0]['name']) for v in results['venues']])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Gym', 
                  'Gym Latitude', 
                  'Gym Longitude', 
                  'Category']
    return(nearby_venues)

In [21]:
gyms_copy = getNearbyGyms(names=data['Borough'], latitudes=data['Latitude'], longitudes=data['Longitude'], r=np.sqrt(table['Area (km2)']/math.pi)*1000)
gyms_copy         #to avoid always making a call to Foursquare while debugging

,Borough,Borough Latitude,Borough Longitude,Gym,Gym Latitude,Gym Longitude,Category
0,Barnet,51.560700,-0.210017,Swiss Cottage Climbing Centre,51.542497,-0.173436,Rock Climbing Spot
1,Barnet,51.560700,-0.210017,Better Swiss Cottage Leisure Centre,51.542550,-0.172956,Gym / Fitness Center
2,Barnet,51.560700,-0.210017,Westway Climbing Wall,51.515455,-0.220350,Gym / Fitness Center
3,Brent,51.454900,-0.267821,Climbing Tree,51.488373,-0.253421,Scenic Lookout
4,Camden,51.546394,-0.157424,Swiss Cottage Climbing Centre,51.542497,-0.173436,Rock Climbing Spot
5,Camden,51.546394,-0.157424,Better Swiss Cottage Leisure Centre,51.542550,-0.172956,Gym / Fitness Center
6,Ealing,51.513000,-0.331026,The Arch Acton,51.506720,-0.265233,Climbing Gym
7,Greenwich,51.489200,0.056244,The Reach,51.494364,0.042757,Gym / Fitness Center
8,Hackney,51.545000,-0.063316,Castle Climbing Centre,51.565339,-0.092413,Climbing Gym
9,Hackney,51.545000,-0.063316,Castle Climbing Centre Cafe,51.565339,-0.092517,Café


In [22]:
#check the different categories of the gyms
gyms=gyms_copy
gyms['Category'].unique()

array(['Rock Climbing Spot', 'Gym / Fitness Center', 'Scenic Lookout',
       'Climbing Gym', 'Café', 'Park', 'Gym'], dtype=object)

In [23]:
#remove the venues which are not climbing gyms
for col in ['Scenic Lookout', 'Café', 'Park']:
    gyms = gyms[gyms.Category !=col]
gyms=gyms.reset_index(drop=True)
gyms

,Borough,Borough Latitude,Borough Longitude,Gym,Gym Latitude,Gym Longitude,Category
0,Barnet,51.560700,-0.210017,Swiss Cottage Climbing Centre,51.542497,-0.173436,Rock Climbing Spot
1,Barnet,51.560700,-0.210017,Better Swiss Cottage Leisure Centre,51.542550,-0.172956,Gym / Fitness Center
2,Barnet,51.560700,-0.210017,Westway Climbing Wall,51.515455,-0.220350,Gym / Fitness Center
3,Camden,51.546394,-0.157424,Swiss Cottage Climbing Centre,51.542497,-0.173436,Rock Climbing Spot
4,Camden,51.546394,-0.157424,Better Swiss Cottage Leisure Centre,51.542550,-0.172956,Gym / Fitness Center
5,Ealing,51.513000,-0.331026,The Arch Acton,51.506720,-0.265233,Climbing Gym
6,Greenwich,51.489200,0.056244,The Reach,51.494364,0.042757,Gym / Fitness Center
7,Hackney,51.545000,-0.063316,Castle Climbing Centre,51.565339,-0.092413,Climbing Gym
8,Hackney,51.545000,-0.063316,Mile End Climbing Wall,51.527822,-0.039800,Climbing Gym
9,Hammersmith and Fulham,51.492700,-0.221290,RavensWall Climbing Centre,51.494347,-0.236619,Climbing Gym


In [24]:
#check that every gym appears only once
gyms['Gym'].value_counts()

The Reach                              2
The Arch Climbing                      2
Swiss Cottage Climbing Centre          2
Stronghold Climbing                    2
Building One                           2
Vauxwall Climbing Centre               2
Better Swiss Cottage Leisure Centre    2
The Climbing Hangar                    2
Vauxwall East Climbing Centre          2
Mile End Climbing Wall                 2
The Arch Acton                         1
White Spider Climbing Wall             1
The Nest Climbing                      1
Westway Climbing Wall                  1
Castle Climbing Centre                 1
RavensWall Climbing Centre             1
Harrowall Climbing Centre              1
Name: Gym, dtype: int64

Due to the imprecise method used to classify each gym in a borough (using the radius), many gyms are assigned to two boroughs. We therefore calculate the distance from each gym to its (two) borough(s) and, in case of duplicates, we assign the gym to the closest one.

In [25]:
from geopy.distance import distance

In [26]:
dist=[]                                #store the distances in a new list "dist"
latlist=[]                             #store the latitudes alredy used in a new list "latlist"

for latB, lngB, latG, lngG in zip(gyms['Borough Latitude'], gyms['Borough Longitude'], gyms['Gym Latitude'], gyms['Gym Longitude']):
    
    cdB= latB, lngB
    cdG= latG, lngB
    
    if cdG[0] in latlist:               #check if the distance of this gym has already been computed
        
        newdist=distance(cdB, cdG).m
        idx=latlist.index(cdG[0])
        
        if newdist<dist[idx]:           #if the new distance is smaller, set the previous one to -1 so that it can be discarded
            dist[idx]=-1
            dist.append(newdist)
        else:
            dist.append(-1)
    else:
        dist.append(distance(cdB, cdG).m)
        
    latlist.append(latG)

gyms['Distance']=dist
gyms

,Borough,Borough Latitude,Borough Longitude,Gym,Gym Latitude,Gym Longitude,Category,Distance
0,Barnet,51.560700,-0.210017,Swiss Cottage Climbing Centre,51.542497,-0.173436,Rock Climbing Spot,-1.000000
1,Barnet,51.560700,-0.210017,Better Swiss Cottage Leisure Centre,51.542550,-0.172956,Gym / Fitness Center,-1.000000
2,Barnet,51.560700,-0.210017,Westway Climbing Wall,51.515455,-0.220350,Gym / Fitness Center,5033.855344
3,Camden,51.546394,-0.157424,Swiss Cottage Climbing Centre,51.542497,-0.173436,Rock Climbing Spot,433.648308
4,Camden,51.546394,-0.157424,Better Swiss Cottage Leisure Centre,51.542550,-0.172956,Gym / Fitness Center,427.778567
5,Ealing,51.513000,-0.331026,The Arch Acton,51.506720,-0.265233,Climbing Gym,698.700377
6,Greenwich,51.489200,0.056244,The Reach,51.494364,0.042757,Gym / Fitness Center,574.480046
7,Hackney,51.545000,-0.063316,Castle Climbing Centre,51.565339,-0.092413,Climbing Gym,2262.908626
8,Hackney,51.545000,-0.063316,Mile End Climbing Wall,51.527822,-0.039800,Climbing Gym,1911.252180
9,Hammersmith and Fulham,51.492700,-0.221290,RavensWall Climbing Centre,51.494347,-0.236619,Climbing Gym,183.204685


In [27]:
#remove the duplicates and clean the dataframe
gyms.drop(gyms[gyms['Distance']==-1].index, inplace=True)
gyms.drop(columns=['Category','Distance'], inplace=True)
gyms=gyms.reset_index(drop=True)
gyms

,Borough,Borough Latitude,Borough Longitude,Gym,Gym Latitude,Gym Longitude
0,Barnet,51.560700,-0.210017,Westway Climbing Wall,51.515455,-0.220350
1,Camden,51.546394,-0.157424,Swiss Cottage Climbing Centre,51.542497,-0.173436
2,Camden,51.546394,-0.157424,Better Swiss Cottage Leisure Centre,51.542550,-0.172956
3,Ealing,51.513000,-0.331026,The Arch Acton,51.506720,-0.265233
4,Greenwich,51.489200,0.056244,The Reach,51.494364,0.042757
5,Hackney,51.545000,-0.063316,Castle Climbing Centre,51.565339,-0.092413
6,Hackney,51.545000,-0.063316,Mile End Climbing Wall,51.527822,-0.039800
7,Hammersmith and Fulham,51.492700,-0.221290,RavensWall Climbing Centre,51.494347,-0.236619
8,Hammersmith and Fulham,51.492700,-0.221290,The Climbing Hangar,51.476383,-0.199611
9,Harrow,51.589800,-0.341267,Harrowall Climbing Centre,51.580914,-0.344160


In [28]:
#sanity check
gyms['Gym'].value_counts()

The Nest Climbing                      1
The Climbing Hangar                    1
The Arch Acton                         1
White Spider Climbing Wall             1
The Arch Climbing                      1
Westway Climbing Wall                  1
Mile End Climbing Wall                 1
Vauxwall East Climbing Centre          1
The Reach                              1
Stronghold Climbing                    1
Better Swiss Cottage Leisure Centre    1
Vauxwall Climbing Centre               1
RavensWall Climbing Centre             1
Building One                           1
Swiss Cottage Climbing Centre          1
Castle Climbing Centre                 1
Harrowall Climbing Centre              1
Name: Gym, dtype: int64

We can now visualise the gyms on the map, together with the boroughs.

In [131]:
for lat, lng, gym in zip(gyms['Gym Latitude'], gyms['Gym Longitude'], gyms['Gym']):
    label = '{}'.format(gym)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(london2)  
    
london2

FINDING THE BEST BOROUGH TO OPEN A NEW CLIMBING GYM

In [98]:
#create a new dataframe containing all the boroughs which do not have a climbing gym
bor=data.loc[~data['Borough'].isin(gyms['Borough'])]
bor

,Borough,Population density (people/km2),Latitude,Longitude
0,Barking and Dagenham,5792.891651,51.545277,0.133528
2,Bexley,4034.402238,51.457389,0.138861
3,Brent,7678.080133,51.454900,-0.267821
4,Bromley,2183.928180,51.371999,0.051524
6,City of London,3033.645891,51.529000,-0.092171
7,Croydon,4466.560887,51.371400,-0.087157
9,Enfield,4050.769646,51.653800,-0.087272
13,Haringey,9398.941098,51.600000,-0.107470
15,Havering,2264.944133,51.581200,0.221108
17,Hounslow,4899.061817,51.474600,-0.367123


In [99]:
#find the five closest boroughs for each borough

def listToString(s):              #function to convert a list to a string
     
    str1 = " " 
    return (str1.join(s))


t5bor=[]

for latB, lngB in zip(bor['Latitude'], bor['Longitude']):
    
    dist=[]
    name=data['Borough'].tolist()
    
    for latG, lngG in zip(data['Latitude'], data['Longitude']):
        
        cdB= latB, lngB
        cdG= latG, lngG
    
        dist.append(distance(cdB, cdG).m)
       
    zipped = zip(dist, name)                    #sort the distances in ascending order, together with the boroughs
    sort = sorted(zipped)
    tuples = zip(*sort)
    dist, name = [ list(tuple) for tuple in  tuples]
    
    t5bor.append(listToString(name[1:6]))       #extract the five closest boroughs (excluding itself) after converting to string
    
bor['Neighbours']=t5bor
bor

C:\Users\marcori94\anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Borough,Population density (people/km2),Latitude,Longitude,Neighbours
0,Barking and Dagenham,5792.891651,51.545277,0.133528,Redbridge Havering Newham Greenwich Bexley
2,Bexley,4034.402238,51.457389,0.138861,Greenwich Newham Barking and Dagenham Lewisham...
3,Brent,7678.080133,51.454900,-0.267821,Richmond upon Thames Hammersmith and Fulham Ki...
4,Bromley,2183.928180,51.371999,0.051524,Lewisham Croydon Bexley Greenwich Newham
6,City of London,3033.645891,51.529000,-0.092171,Islington Hackney Southwark Tower Hamlets Camden
7,Croydon,4466.560887,51.371400,-0.087157,Sutton Merton Lewisham Bromley Lambeth
9,Enfield,4050.769646,51.653800,-0.087272,Haringey Waltham Forest Hackney Islington Camden
13,Haringey,9398.941098,51.600000,-0.107470,Enfield Islington Waltham Forest Hackney Camden
15,Havering,2264.944133,51.581200,0.221108,Barking and Dagenham Redbridge Bexley Newham G...
17,Hounslow,4899.061817,51.474600,-0.367123,Richmond upon Thames Ealing Brent Kingston upo...


In [100]:
#check whether any of the neighbouring boroughs has a gym and, in case one has, the borough is discarded
match = gyms['Borough'].tolist()
for i in range(len(match)):
    if i==0:
        rst=bor.loc[~bor['Neighbours'].str.contains(match[i])]
    else:
        rst=rst.loc[~rst['Neighbours'].str.contains(match[i])]
rst

,Borough,Population density (people/km2),Latitude,Longitude,Neighbours
7,Croydon,4466.560887,51.3714,-0.087157,Sutton Merton Lewisham Bromley Lambeth


We arrive at our final result: Croydon is the only London borough without a climbing gym whose five closest boroughs also lack a gym.

In [136]:
folium.Marker(
    location=[rst['Latitude'][7], rst['Longitude'][7]],
    popup='Gym',
    icon=folium.Icon(color='green')
).add_to(london2)  
    
london2